In [1]:
import ScraperFC as sfc
import traceback
import pandas as pd
pd.options.display.max_columns = 50
from IPython.display import clear_output

In [2]:
# # scraper = sfc.Understat()
# scraper = sfc.FBRef()

# error = False
# shapes = []
# for yr in range(2018,2021): #1988,2021):
#     for lg in ['EPL','La Liga','Bundesliga','Serie A','Ligue 1']:

#         try:
#         #     out = scraper.scrape_situations(2019, "La Liga")
#         #     out = scraper.scrape_formations(2020, "EPL")
#         #     out = scraper.scrape_game_states(2020, "EPL")

#             out = scraper.scrape_matches(yr, lg)
#             shapes.append(
#                 [lg, yr, out.shape, out["Home Player Stats"], out["Away Player Stats"]]
#             )
#     #         scraper.close()
#         except:  
#             traceback.print_exc()
#             scraper.close()
#             error = True
#             break

#     if error:
#         break
#     scraper.close()

In [3]:
scraper = sfc.Understat()

try:
    out = scraper.scrape_shot_zones(2020, "Ligue 1")
    scraper.close()
except:  
    traceback.print_exc()
    scraper.close()

['Toulouse' 1 1 2 2 1.0 2.0 -1.0 1.0 1.0 127 3 162 11 4.14 5.49 -1.35 0.03
 0.03 167 15 222 36 21.48 33.01 -11.53 0.13 0.15 22 3 28 9 5.79 10.03
 -4.25 0.26 0.36]
(1, 37)
['Lyon' 0 0 3 3 0.0 3.0 -3.0 0.0 1.0 156 10 160 3 4.84 4.41 0.43 0.03 0.03
 179 25 137 19 24.07 16.9 7.18 0.13 0.12 25 7 12 2 7.49 3.56 3.93 0.3 0.3]
(1, 37)
['Monaco' 0 0 1 1 0.0 1.0 -1.0 0.0 1.0 122 1 136 5 4.38 3.9 0.48 0.04 0.03
 228 30 202 27 32.91 26.61 6.3 0.14 0.13 28 13 30 11 12.36 10.79 1.57 0.44
 0.36]
(1, 37)
['Metz' 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 154 3 169 7 4.26 5.27 -1.0
 0.03 0.03 142 22 192 19 17.37 23.21 -5.83 0.12 0.12 14 2 27 9 3.05 8.11
 -5.06 0.22 0.3]
(1, 37)
['Nice' 1 1 2 2 1.0 2.0 -1.0 1.0 1.0 145 6 143 3 4.63 3.9 0.72 0.03 0.03
 169 25 218 26 23.45 29.43 -5.98 0.14 0.13 25 9 30 7 9.34 9.67 -0.34 0.37
 0.32]
(1, 37)
['Rennes' 2 2 1 1 2.0 1.0 1.0 1.0 1.0 154 5 169 3 5.26 4.67 0.59 0.03 0.03
 167 23 166 11 22.11 17.2 4.91 0.13 0.1 22 8 23 9 8.02 6.51 1.51 0.36 0.28]
(1, 37)
['Dijon' 1 1 2 2

In [4]:
out

Team names Own goals                                    \
                   Team        Sh  G ShA GA xG xGA xGD xG/Sh xGA/Sh   
0              Toulouse         1  1   2  2  1   2  -1     1      1   
1                  Lyon         0  0   3  3  0   3  -3     0      1   
2                Monaco         0  0   1  1  0   1  -1     0      1   
3                  Metz         0  0   0  0  0   0   0     0      0   
4                  Nice         1  1   2  2  1   2  -1     1      1   
5                Rennes         2  2   1  1  2   1   1     1      1   
6                 Dijon         1  1   2  2  1   2  -1     1      1   
7            Strasbourg         0  0   0  0  0   0   0     0      0   
8                Angers         1  1   0  0  1   0   1     1      0   
9              Bordeaux         0  0   1  1  0   1  -1     0      1   
10               Amiens         0  0   1  1  0   1  -1     0      1   
11  Paris_Saint_Germain         4  4   1  1  4   1   3     1      1   
12                Brest         2  2   0  0  2   0   2     1      0   
13                Lille         0  0   3  3  0   3  -3     0      1   
14            Marseille         3  3   3  3  3   3   0     1      1   
15        Saint-Etienne         1  1   0  0  1   0   1     1      0   
16                Reims         0  0   0  0  0   0   0     0      0   
17               Nantes         5  5   0  0  5   0   5     1      0   
18          Montpellier         2  2   2  2  2   2   0     1      1   
19                Nimes         0  0   1  1  0   1  -1     0      1   

   Out of box                                             Penalty area      \
           Sh   G  ShA  GA    xG   xGA   xGD xG/Sh xGA/Sh           Sh   G   
0         127   3  162  11  4.14  5.49 -1.35  0.03   0.03          167  15   
1         156  10  160   3  4.84  4.41  0.43  0.03   0.03          179  25   
2         122   1  136   5  4.38   3.9  0.48  0.04   0.03          228  30   
3         154   3  169   7  4.26  5.27    -1  0.03   0.03          142  22   
4         145   6  143   3  4.63   3.9  0.72  0.03   0.03          169  25   
5         154   5  169   3  5.26  4.67  0.59  0.03   0.03          167  23   
6         134   1  160   8  3.87  5.43 -1.56  0.03   0.03          190  23   
7         124   2  104   3  4.05  2.85  1.21  0.03   0.03          181  22   
8         170   6  103   4  5.13  3.19  1.94  0.03   0.03          177  17   
9         151  13  155   5  5.39  4.53  0.85  0.04   0.03          139  24   
10        101   2  189  10   2.9  5.44 -2.54  0.03   0.03          121  22   
11        119   5  108   2  4.51  2.95  1.56  0.04   0.03          275  52   
12        125   6  145   4   3.4  5.91 -2.51  0.03   0.04          182  23   
13        129   3  100   3  4.04  3.21  0.83  0.03   0.03          208  28   
14        174   6  121   2  5.63  3.97  1.66  0.03   0.03          183  24   
15        125   1  130   3  3.76  4.28 -0.52  0.03   0.03          165  23   
16        129   6  128   2  3.67  3.57   0.1  0.03   0.03          185  18   
17        159   3  118   5   4.2  3.97  0.22  0.03   0.03          160  17   
18        151   9  180   6  4.43  6.36 -1.93  0.03   0.04          173  17   
19        166   3  135   5   4.9  4.09  0.81  0.03   0.03          167  20   

                                              Six-yard box                     \
    ShA  GA     xG    xGA    xGD xG/Sh xGA/Sh           Sh   G ShA  GA     xG   
0   222  36  21.48  33.01 -11.53  0.13   0.15           22   3  28   9   5.79   
1   137  19  24.07   16.9   7.18  0.13   0.12           25   7  12   2   7.49   
2   202  27  32.91  26.61    6.3  0.14   0.13           28  13  30  11  12.36   
3   192  19  17.37  23.21  -5.83  0.12   0.12           14   2  27   9   3.05   
4   218  26  23.45  29.43  -5.98  0.14   0.13           25   9  30   7   9.34   
5   166  11  22.11   17.2   4.91  0.13    0.1           22   8  23   9   8.02   
6   215  22  20.89  26.56  -5.67  0.11   0.12           12   2  21   5   2.88   
7   